In [ ]:
from __future__ import print_function
import os
import sys
import timeit
import numpy
import theano
import theano.tensor as T
from theano.tensor.shared_randomstreams import RandomStreams

try:
    import PIL.Image as Image
except ImportError:
    import Image

class DenoisingAutoencoder:
    
    def __init__(self, numpy_rng, theano_rng = None, input = None, n_visible = 784, n_hidden = 500, 
                 W = None, bhid = None, bvis = None):
        
        self.n_visible = n_visible
        self.n_hidden = n_hidden
        
        # make a Theano random generator which gives symbolic random values
        if not theano_rng:
            theano_rng = RandomStreams(numpy_rng.randint(2**30))
            
        if not W:
            initial_W = numpy.asarray(
                numpy_rng.uniform(
                    low  = -4 * numpy.sqrt(6. / (n_hidden + n_visible)),
                    high =  4 * numpy.sqrt(6. / (n_hidden + n_visible)),
                    size = (n_visible, n_hidden)
                ),
                dtype = theano.config.floatX
            )
            W = theano.shared(value = initial_W, name = 'W', borrow = True)
            
        if not bvis:
            bvis = theano.shared(
                value = numpy.zeros(
                    n_visible,
                    dtype = theano.config.floatX
                ),
                borrow = True
            )
        
        if not bhid:
            bhid = theano.shared(
                value = numpy.zeros(
                    n_hidden,
                    dtype = theano.config.floatX
                ),
                name = 'b',
                borrow = True
            )
        
        self.W = W
        
        # b is the bias of the hidden
        self.b = bhid
        
        # b_prime is the bias of the visible
        self.b_prime = bvis
        
        self.W_prime = self.W.T
        self.theano_rng = theano_rng
        
        # if there is no input, make a variable representing input
        if input is None:
            self.x = T.dmatrix(name = 'input')
        else:
            self.x = input
        
        self.params = [self.W, self.b, self.b_prime]
        
    def get_corrupted_input(self, input, corruption_level):
        return self.theano_rng.binomial(size = input.shape, n = 1,
                                        p = 1 - corruption_level,
                                        dtype = theano.config.floatX) * input
    
    def get_hidden_values(self, input):
        # get the values of the hidden layer
        return T.nnet.sigmoid(T.dot(input, self.W) + self.b)
    
    def get_reconstructed_input(self, hidden):
        # finds the reconstructed input given the hidden layer's values
        return T.nnet.sigmoid(T.dot(hidden, self.W_prime) + self.b_prime)
    
    def get_cost_updates(self, corruption_level, learning_rate):
        # computes the cost and the updates for one training step
        
        tilde_x = self.get_corrupted_input(self.x, corruption_level)
        y = self.get_hidden_values(tilde_x)
        z = self.get_reconstructed_input(y)
        L = -T.sum(self.x * T.log(z) + (1 - self.x) * T.log(1 - z), axis = 1)
        
        cost = T.mean(L)
        
        # compute the gradients of the cost of the auto-encoder with respect to its parameters
        gparams = T.grad(cost, self.params)
        
        # make the list of updates
        updates = [
            (param, param - learning_rate * gparam)
            for param, gparam in zip(self.params, gparams)
        ]
        
        return (cost, updates)